## Model Saving Example

This notebook demonstrates saving and loading a tensorflow model into and out of a class.

It implements a class to store the basic skeleton of a tensorflow model, but leaves out any training or prediction methods.

In [1]:
import tensorflow as tf
import numpy as np

### Model Class

We can use classes to encapsulate tensorflow models. The below skeleton shows one way of using classes with tensorflow.

The Model class has methods, variables, and properties that capture both the graph and the tensorflow session

#### Tensorflow Graph

A tensorflow graph is a computational graph of different tensorflow operations. It defines the computation and how different operations and tensors relate, but it doesn't actually do the computation or store the values of the variables. All of that magic happens within the tensorflow session.

#### Tensorflow Session

A tensorflow session is the context where values for tensorflow variables are instantiated and computations are run. So if you are saving a model's weights, you are actually saving the weights of the tensorflow session. If you are loading a model's weights, you need to load them into a session. When variables are initialized, that has to happen within a session. In a way, the graph is stateless. State is stored in sessions. The session also takes care of running computations, so if you are running training, those need to be run in the session.

A session is instantiated with a graph, typically the current default graph. A session is only able to run computations on the graph that is tied to the session.

In [5]:
class Model():
    """
    Tutorial Model
    """
    
    def __init__(self):
        self._graph = None
        self._session = None
    
    
    def load_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
        
        self._session = tf.Session(graph=self.graph)
        model_saver.restore(self._session, model_filename)
        return
    
    
    def save_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
            
        model_saver.save(self.session, model_filename)
    
    
    def create_graph(self):
        self._graph = tf.Graph()
        with self._graph.as_default():
            self.weights = tf.Variable(np.random.rand(10, 6).astype(np.float32), dtype=tf.float32, name="weights")
            self.weights_assignment = tf.placeholder(tf.float32, name="weights_assignment")
            self.assign_weights_op = self.weights.assign(self.weights_assignment)
            self._init_op = tf.global_variables_initializer()
        return
    
    def init_model(self):
        self.session.run(self._init_op)
    
    def assign_weights(self, value):
        self.session.run(self.assign_weights_op, 
                        feed_dict={self.weights_assignment: value})
    
    @property
    def graph(self):
        if self._graph is None:
            self.create_graph()
        return self._graph
    
    @property
    def session(self):
        if self._session is None:
            self._session = tf.Session(graph=self.graph)
        return self._session

In [18]:
# create a model instance with 2 hidden layers and 10 hidden units.

model_a = Model()

### Lazy Properties

The class doesn't actually create the graph or session until the graph and session properties are called. The @property decorator functions above are used to create a graph or session if none exists.

In [19]:
model_a._graph is None

True

In [20]:
model_a._session is None

True

In [21]:
model_a.weights

AttributeError: Model instance has no attribute 'weights'

In [22]:
model_a.graph

In [23]:
model_a.session

In [24]:
model_a.weights

<tf.Variable 'weights:0' shape=(10, 6) dtype=float32_ref>

In [25]:
model_a.init_model()

In [27]:
model_a.session.run(model_a.weights)

array([[ 0.68956363,  0.24255228,  0.39409241,  0.61739135,  0.20062552,
         0.66500258],
       [ 0.18648416,  0.39518887,  0.4174217 ,  0.24635126,  0.13605003,
         0.21980985],
       [ 0.18645693,  0.70708805,  0.6880247 ,  0.86992157,  0.37679633,
         0.93836826],
       [ 0.93225998,  0.48493031,  0.10632242,  0.92587602,  0.31251162,
         0.34510398],
       [ 0.06108605,  0.98616314,  0.04361878,  0.58846837,  0.73885441,
         0.02847404],
       [ 0.05655411,  0.87358284,  0.91675007,  0.43365577,  0.74119973,
         0.75215054],
       [ 0.86880887,  0.45104972,  0.02854795,  0.21982266,  0.22426881,
         0.95298648],
       [ 0.15122464,  0.09607223,  0.16057616,  0.91191608,  0.02949475,
         0.26094165],
       [ 0.65045106,  0.69455224,  0.863859  ,  0.59957445,  0.54131889,
         0.67963868],
       [ 0.02727904,  0.66972971,  0.9377073 ,  0.39222211,  0.99323404,
         0.73217767]], dtype=float32)

### Assignment Operations

The class adds some convenience functions for assigning weights. Tensorflow can only assign values to tensor variables using assignment operations, and a combination of a placeholder and assignment operation are used to allow the assignment through a function.

In [32]:
model_a.assign_weights(np.ones((10, 6)))

In [33]:
model_a.session.run(model_a.weights)

array([[ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.]], dtype=float32)

In [34]:
model_a.save_model("./test_saved_model.cpkt")

### Load the saved model into a second model

In [35]:
model_b = Model()

In [36]:
model_b.load_model("./test_saved_model.cpkt")

INFO:tensorflow:Restoring parameters from ./test_saved_model.cpkt


In [37]:
model_b.weights

<tf.Variable 'weights:0' shape=(10, 6) dtype=float32_ref>

In [39]:
model_b.session.run(model_b.weights)

array([[ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.]], dtype=float32)